# Import modules

In [1]:
from ntpath import join
import math
import re

import geopandas as gpd
import pandas as pd
import rasterio as rio
import morecantile
import pyproj

import requests
import lxml
import os, sys
import argparse
from tqdm import tqdm
import yaml
import json

import shutil
import time

from shapely.geometry import box
from osgeo import gdal
from cogeo_mosaic.mosaic import MosaicJSON

os.chdir('..')

/home/gsalamin/Python_env/road_surfaces/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
with open('03_Scripts/config.yaml') as fp:
    cfg = yaml.load(fp, Loader=yaml.FullLoader)['prepare_data.py']    #  [os.path.basename(__file__)]


# Task to do
DETERMINE_ROAD_SURFACES = cfg['tasks']['determine_roads_surfaces']
DETERMINE_RESTRICTED_AOI = cfg['tasks']['determine_restricted_AOI']
MAKE_RASTER_MOSAIC = cfg['tasks']['make_raster_mosaic']
GENERATE_TILES_JSON=cfg['tasks']['generate_tiles_info']

if not DETERMINE_ROAD_SURFACES and not DETERMINE_RESTRICTED_AOI and not MAKE_RASTER_MOSAIC and not GENERATE_TILES_JSON:
    print('Nothing to do. Exiting!')
    sys.exit(0)
else:
    
    DEBUG_MODE = cfg['debug_mode']

    INPUT = cfg['input']
    INPUT_DIR =INPUT['input_folder']
    IM_FOLDER_IN= INPUT['images_folder_in']

    ROADS_IN = INPUT_DIR + INPUT['input_files']['roads']
    ROADS_PARAM = INPUT_DIR + INPUT['input_files']['roads_param']
    FORESTS = INPUT_DIR + INPUT['input_files']['forests']
    

    OUTPUT = cfg['output']
    OUTPUT_DIR = OUTPUT['output_folder']

    if DETERMINE_ROAD_SURFACES:
        ROADS_OUT = OUTPUT_DIR +  OUTPUT['output_files']['roads']

    if DETERMINE_RESTRICTED_AOI:
        RESTRICTED_AOI = OUTPUT_DIR +  OUTPUT['output_files']['restricted_AOI']
    
    if MAKE_RASTER_MOSAIC:
        IMAGES_BOUNDS = INPUT_DIR + INPUT['input_files']['images_bounds']
        IM_FOLDER_OUT = OUTPUT_DIR +  OUTPUT['output_files']['images_folder_out']
        MOSAIC =  OUTPUT_DIR +  OUTPUT['output_files']['mosaic']
    
    if GENERATE_TILES_JSON:
        TILES_AOI = OUTPUT_DIR + OUTPUT['output_files']['tiles_aoi']
        GPKG_TILES = OUTPUT_DIR + OUTPUT['output_files']['gpkg_tiles']
        ZOOM_LEVEL = cfg['param_tiles']['zoom_level']



# Define functions

In [3]:
def polygons_diff_without_artifacts(polygons, p1_idx, p2_idx):
    # Make the difference of the geometry at row p2_idx with the one at the row p1_idx
    
    diff=polygons.loc[p2_idx,'geometry']-polygons.loc[p1_idx,'geometry']

    # Store intermediary results back to poly
    if diff.geom_type == 'Polygon':
        polygons.loc[p2_idx,'geometry'] -= polygons.loc[p1_idx,'geometry']

    elif diff.geom_type == 'MultiPolygon':
        # if a multipolygone is created, only keep the largest part to avoid the following error: https://github.com/geopandas/geopandas/issues/992
        polygons.loc[p2_idx,'geometry'] = max((polygons.loc[p2_idx,'geometry']-polygons.loc[p1_idx,'geometry']).geoms, key=lambda a: a.area)

    return polygons

def test_crs(crs1,crs2 = "EPSG:2056"):
    try:
        assert(crs1 == crs2), "CRS mismatch between the files."
    except Exception as e:
        print(e)
        sys.exit(1)

# Main
## Import files

In [4]:
# Import files ------------------------------------------------------------------------------------------
print('Importing files...')
## Data
roads=gpd.read_file(ROADS_IN)
forests=gpd.read_file(FORESTS)

## Other informations
roads_parameters=pd.read_excel(ROADS_PARAM)

print('Importations done!')

Importing files...
Importations done!


## Creation of the polygons for the roads

### Filter the roads to work on

In [11]:
# Supress non-roads elements
roads_parameters=roads_parameters[roads_parameters['to keep']=='yes']
roads_parameters.drop_duplicates(subset='GDB-Code',inplace=True)       # Keep first by default 

joined_roads=roads.merge(roads_parameters[['GDB-Code','Width']], how='right',left_on='OBJEKTART',right_on='GDB-Code')

### Buffer the roads

In [6]:
# Buffer the roads
buffered_roads=joined_roads.copy()
buffered_roads['buffered_geom']=buffered_roads.buffer(joined_roads['Width'], cap_style=2)


In [7]:
buffered_roads.drop(columns=['geometry'],inplace=True)
buffered_roads.rename(columns={'buffered_geom':'geometry'},inplace=True)

In [8]:
## Do not let roundabouts make artifacts
for idx in buffered_roads.index:
    geom=buffered_roads.loc[idx,'geometry']
    if geom.geom_type == 'MultiPolygon':
        buffered_roads.loc[idx,'geometry'] = max(buffered_roads.loc[idx,'geometry'].geoms, key=lambda a: a.area)
        print(f'ID: {buffered_roads.loc[idx,"OBJECTID"]}')

ID: 314.0
ID: 315.0
ID: 373.0
ID: 374.0
ID: 375.0
ID: 376.0
ID: 1580.0
ID: 1581.0
ID: 1582.0
ID: 25294.0
ID: 25297.0
ID: 25298.0
ID: 25299.0
ID: 26669.0
ID: 1576.0
ID: 1577.0
ID: 1578.0


In [9]:
buffered_roads.shape

(11565, 32)

### Erease overlapping zones of polygons

In [10]:
buffered_roads['saved_geom']=buffered_roads.geometry
joined_roads=gpd.sjoin(buffered_roads,buffered_roads[['OBJECTID','OBJEKTART','saved_geom','geometry']],how='left', lsuffix='1', rsuffix='2')

### Drop excessive rows
intersected=joined_roads[joined_roads['OBJECTID_2'].notna()].copy()
intersected_not_itself=intersected[intersected['OBJECTID_1']!=intersected['OBJECTID_2']].copy()
intersected_roads=intersected_not_itself.drop_duplicates(subset=['OBJECTID_1','OBJECTID_2'])

intersected_roads.reset_index(inplace=True, drop=True)
print(intersected_roads.shape)

(30476, 37)


In [11]:
### Sort the roads so that the widest ones come first
intersected_roads.loc[intersected_roads['OBJEKTART_1']==20,'OBJEKTART_1']=8.5

intersect_other_width=intersected_roads[intersected_roads['OBJEKTART_1']<intersected_roads['OBJEKTART_2']].copy()

intersect_other_width.sort_values(by=['OBJEKTART_1'],inplace=True)
intersect_other_width.loc[intersect_other_width['OBJEKTART_1']==8.5,'OBJEKTART_1']=20

intersect_other_width.reset_index(inplace=True, drop=True)

print(intersect_other_width.shape)

(3478, 37)


In [19]:
### Suppress the overlapping intersection
# from https://stackoverflow.com/questions/71738629/expand-polygons-in-geopandas-so-that-they-do-not-overlap-each-other

corr_overlap1 = buffered_roads.copy()

for idx in tqdm(intersect_other_width.index, total=intersect_other_width.shape[0],
               desc='Suppressing the overlap of roads with different width'):
    
    poly1_id = corr_overlap1.index[corr_overlap1['OBJECTID'] == intersect_other_width.loc[idx,'OBJECTID_1']].values.astype(int)[0]
    poly2_id = corr_overlap1.index[corr_overlap1['OBJECTID'] == intersect_other_width.loc[idx,'OBJECTID_2']].values.astype(int)[0]
    
    corr_overlap1=polygons_diff_without_artifacts(corr_overlap1,poly1_id,poly2_id)

Supressing the overlapping intersections of roads of different width:: 100%|█| 3


In [20]:
corr_overlap1.drop(columns=['saved_geom'],inplace=True)

In [21]:
corr_overlap1.columns

Index(['OBJECTID', 'UUID', 'DATUM_AEND', 'DATUM_ERST', 'ERSTELLUNG',
       'ERSTELLU_1', 'REVISION_J', 'REVISION_M', 'GRUND_AEND', 'HERKUNFT',
       'HERKUNFT_J', 'HERKUNFT_M', 'REVISION_Q', 'OBJEKTART', 'KUNSTBAUTE',
       'WANDERWEGE', 'VERKEHRSBE', 'BEFAHRBARK', 'EROEFFNUNG', 'STUFE',
       'RICHTUNGSG', 'BELAGSART', 'KREISEL', 'EIGENTUEME', 'VERKEHRS_1',
       'NAME', 'TLM_STRASS', 'STRASSENNA', 'SHAPE_Leng', 'GDB-Code', 'Width',
       'geometry'],
      dtype='object')

### Remove overlapping area between roads of the same class

In [22]:
## Remove overlapping area between roads of the same class

save_geom=corr_overlap1.copy()
save_geom['saved_geom']=save_geom.geometry
joined_roads=gpd.sjoin(save_geom,save_geom[['OBJECTID','saved_geom','geometry']],how='left', lsuffix='1', rsuffix='2')

### Drop excessive rows
intersected=joined_roads[joined_roads['OBJECTID_2'].notna()].copy()
intersected_not_itself=intersected[intersected['OBJECTID_1']!=intersected['OBJECTID_2']].copy()
intersected_roads=intersected_not_itself.drop_duplicates(subset=['OBJECTID_1','OBJECTID_2'])

intersected_roads.reset_index(inplace=True, drop=True)
print(intersected_roads.shape)

(29656, 36)


In [24]:
### Get rid of duplicates with inverted objectid
to_drop=[]
for idx in tqdm(intersected_roads.index, total=intersected_roads.shape[0],
               desc='Erease duplicates from spatial join'):
    ir1_objid=intersected_roads.loc[idx,'OBJECTID_1']
    ir2_objid=intersected_roads.loc[idx,'OBJECTID_2']
    
    for ss_idx in intersected_roads[intersected_roads['OBJECTID_1']==ir2_objid].index:
        
        if ir1_objid==intersected_roads.loc[ss_idx,'OBJECTID_2'] and idx<ss_idx:
            to_drop.append(ss_idx)


Erease duplicates from spatial join:: 100%|█| 29656/29656 [00:32<00:00, 913.92it


In [25]:
intersected_roads.drop(to_drop,inplace=True)

In [26]:
corr_overlap2=corr_overlap1.copy()

### from https://stackoverflow.com/questions/71738629/expand-polygons-in-geopandas-so-that-they-do-not-overlap-each-other
for idx in tqdm(intersected_roads.index, total=intersected_roads.shape[0],
                desc='Suppressing overlap between equivalent roads'):
    
    poly1_id = corr_overlap2.index[corr_overlap2['OBJECTID'] == intersected_roads.loc[idx,'OBJECTID_1']].values.astype(int)[0]
    poly2_id = corr_overlap2.index[corr_overlap2['OBJECTID'] == intersected_roads.loc[idx,'OBJECTID_2']].values.astype(int)[0]
    
    geom1 = corr_overlap2.loc[poly1_id,'geometry']
    geom2 = corr_overlap2.loc[poly2_id,'geometry']

    # Store intermediary results in variable
    diff = geom2 - geom1
    
    if diff.geom_type == 'Polygon':
        temp = geom2 - geom1
        
    elif diff.geom_type == 'MultiPolygon':
        # if a multipolygone is created, only keep the largest part to avoid the following error: https://github.com/geopandas/geopandas/issues/992
        temp = max((geom2 - geom1).geoms, key=lambda a: a.area)

    corr_overlap2=polygons_diff_without_artifacts(corr_overlap2,poly2_id,poly1_id)

    corr_overlap2.loc[poly2_id,'geometry']=temp

Suppressing overlap between equivalent roads: 100%|█| 14828/14828 [00:12<00:00, 


In [27]:
corr_overlap2.columns

Index(['OBJECTID', 'UUID', 'DATUM_AEND', 'DATUM_ERST', 'ERSTELLUNG',
       'ERSTELLU_1', 'REVISION_J', 'REVISION_M', 'GRUND_AEND', 'HERKUNFT',
       'HERKUNFT_J', 'HERKUNFT_M', 'REVISION_Q', 'OBJEKTART', 'KUNSTBAUTE',
       'WANDERWEGE', 'VERKEHRSBE', 'BEFAHRBARK', 'EROEFFNUNG', 'STUFE',
       'RICHTUNGSG', 'BELAGSART', 'KREISEL', 'EIGENTUEME', 'VERKEHRS_1',
       'NAME', 'TLM_STRASS', 'STRASSENNA', 'SHAPE_Leng', 'GDB-Code', 'Width',
       'geometry'],
      dtype='object')

In [28]:
# corr_overlap1.to_file("/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/test_buffer1.shp")

# corr_overlap2.to_file("/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/test_buffer2.shp")

### Make the difference with the forested area

In [29]:
# Do not consider the roads under forest canopy
test_crs(corr_overlap2.crs, forests.crs)

In [30]:
non_forest_roads=corr_overlap2.copy()
non_forest_roads=non_forest_roads.overlay(forests[['UUID','geometry']],how='difference')

# Make a vector of the zones to keep on the pictures

In [ ]:
width=roads_parameters['Width'].max()+1

buffered_roads_aoi=joined_roads.copy()
buffered_roads_aoi['buffered_geom']=buffered_roads_aoi.buffer(width)

buffered_roads_aoi.drop(columns=['geometry'],inplace=True)
buffered_roads_aoi.rename(columns={'buffered_geom':'geometry'},inplace=True)

AOI_roads=buffered_roads_aoi.unary_union


In [ ]:
test_crs(AOI_roads.crs, forests.crs)
    
geom={'geometry':[x for x in AOI_roads.geoms]}
AOI_roads_no_forest=gpd.GeoDataFrame(geom, crs=roads.crs)
AOI_roads_no_forest=AOI_roads_no_forest.overlay(forests[['UUID','geometry']],how='difference')

# Make the raster mosaic

In [5]:
dataset = []

if DEBUG_MODE:
    files=os.listdir(IM_FOLDER_IN)[0:40]
else:
    files=os.listdir(IM_FOLDER_IN)

for f in files:
    if f.endswith('.tif'):
        dataset.append(os.path.join(IM_FOLDER_IN, f))

In [6]:
t1 = time.time()

if False:
    ## copy images on the local disk
    for f in tqdm(dataset, desc="1st step: copy + overviews"):

        src_filename = f
        dst_filename = os.path.join(IM_FOLDER_OUT, os.path.basename(f))

        # test if the tif file already exists
        if os.path.isfile(dst_filename):
            continue

        # copy + generate overviews
        src_ds = gdal.Open(src_filename)
        driver = gdal.GetDriverByName('GTiff')
        dst_ds = driver.CreateCopy(dst_filename, src_ds, strict=0) #, options=['TILED=YES', 'COPY_SRC_OVERVIEWS=YES', 'COMPRESS=DEFLATE'])

        dst_ds.BuildOverviews("AVERAGE", [2, 4, 8, 16, 32, 64, 128, 256])
        epsg2056 = pyproj.CRS.from_epsg(2056)
        dst_ds.SetProjection(epsg2056.to_wkt())

        # Once we're done, close properly the dataset
        dst_ds = None
        src_ds = None

    
## collect statistics
stats = {}

for f in tqdm(dataset, desc="2nd step: collecting statistics"):

    file_basename = os.path.basename(f)

    src_filename = os.path.join(IM_FOLDER_IN, os.path.basename(f))

    src_ds = gdal.Open(src_filename)

    stats[file_basename] = {}
    for i in range(1, src_ds.RasterCount+1):
        # cf. https://arijolma.org/Geo-GDAL/2.4/classGeo_1_1GDAL_1_1Band.html#a8e2361c3a2d76bbfa7253370c78990cf
        stats[file_basename][i] = src_ds.GetRasterBand(i).GetStatistics(1, 1)
        
        
# analyze stats
rgb_stats = {k: {2: v[2], 3: v[3], 4: v[4]} for k, v in stats.items()}
nir_stats = {k: {1: v[1]} for k, v in stats.items()}

rgb_stats_list = [list(x.values()) for x in rgb_stats.values()]
rgb_flat_stats_list = [e for v in rgb_stats_list for e in v]

nir_stats_list = [list(x.values()) for x in nir_stats.values()]
nir_flat_stats_list = [e for v in nir_stats_list for e in v]

rgb_mins = [x[0] for x in rgb_flat_stats_list]
rgb_maxs = [x[1] for x in rgb_flat_stats_list]

nir_mins = [x[0] for x in nir_flat_stats_list]
nir_maxs = [x[1] for x in nir_flat_stats_list]

rgb_min = min(rgb_mins)
rgb_max = max(rgb_maxs)

nir_min = min(nir_mins)
nir_max = max(nir_maxs)

print(f"RGB (min, max) = {(rgb_min, rgb_max)} -- NIR (min, max) = {(nir_min, nir_max)}")


for f in tqdm(dataset, desc="3rd step: COG generation:"):
    # generate COG
    
    # Test if the cog file already exists
    cog_file = os.path.join(IM_FOLDER_OUT, os.path.basename('cog_' + f))
    if os.path.isfile(cog_file):
        continue

    src_filename = os.path.join(IM_FOLDER_IN, os.path.basename(f))
    dst_filename = os.path.join(IM_FOLDER_OUT, f"cog_{os.path.basename(f)}")

    src_ds = gdal.Open(src_filename)
    # scaleParams=[[the_min, the_max, 0, 255]]

    # doc: https://gdal.org/api/python/osgeo.gdal.html?highlight=translate#osgeo.gdal.Translate
    dst_ds = gdal.Translate(
        dst_filename, 
        src_ds, 
        outputType=gdal.GDT_Byte, 
        creationOptions=["TILED=YES", "COPY_SRC_OVERVIEWS=YES", "COMPRESS=DEFLATE"], 
        scaleParams=[[nir_min, nir_max, 0, 65535], [rgb_min, rgb_max, 0, 255], [rgb_min, rgb_max, 0, 255], [rgb_min, rgb_max, 0, 255]]
    )

    # Once we're done, close properly the dataset
    dst_ds = None
    src_ds = None
    
t2 = time.time()

print(f"Elapsed time = {t2-t1:.2f}")

2nd step: collecting statistics: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:12<00:00,  1.03it/s]


RGB (min, max) = (0.0, 65441.0) -- NIR (min, max) = (0.0, 49675.0)


3rd step: COG generation:: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [03:06<00:00, 14.31s/it]

Elapsed time = 198.64


In [9]:
out_dataset = []

for f in os.listdir(IM_FOLDER_OUT):
    if f.startswith('cog_') and f.endswith('.tif'):
        #_el = f"http://localhost:3000/{f}"
        _el = os.path.join(IM_FOLDER_OUT, f)
        out_dataset.append(_el)

try:
    del mosaic_definition
except:
    pass


mosaic_definition = MosaicJSON.from_urls(out_dataset, minzoom=0, maxzoom=24)

/home/gsalamin/Python_env/road_surfaces/lib/python3.8/site-packages/rio_tiler/io/base.py:66: UserWarning: Cannot dertermine bounds in geographic CRS, will default to (-180.0, -90.0, 180.0, 90.0).
  warnings.warn(
/home/gsalamin/Python_env/road_surfaces/lib/python3.8/site-packages/rio_tiler/io/cogeo.py:149: NoOverviewWarning: The dataset has no Overviews. rio-tiler performances might be impacted.
  warnings.warn(
/home/gsalamin/Python_env/road_surfaces/lib/python3.8/site-packages/mercantile/__init__.py:77: FutureWarning: Mercantile 2.0 will require tile x and y to be within the range (0, 2 ** zoom)
  warnings.warn(


In [15]:
with open(MOSAIC, 'w') as fp:
    
    # "hack"
    original_def = mosaic_definition.dict()
    modified_def = json.loads(json.dumps(mosaic_definition.dict()).replace(IM_FOLDER_OUT, "/data"))
    
    json.dump(modified_def, fp)

# Get the information of the tiles for the aoi

In [4]:
if not DETERMINE_RESTRICTED_AOI:
    AOI_roads_no_forest = gpd.read_file(OUTPUT_DIR +  OUTPUT['output_files']['restricted_AOI'])

In [5]:
bboxes_extent_4326=AOI_roads_no_forest.to_crs(epsg=4326).unary_union.bounds


In [6]:
# cf. https://developmentseed.org/morecantile/usage/

tms = morecantile.tms.get("WebMercatorQuad")

epsg3857_tiles_gdf = gpd.GeoDataFrame.from_features([tms.feature(x, projected=True) for x in tqdm(tms.tiles(*bboxes_extent_4326, zooms=[ZOOM_LEVEL]))])
epsg3857_tiles_gdf.set_crs(epsg=3857, inplace=True)


0it [00:00, ?it/s]/home/gsalamin/Python_env/road_surfaces/lib/python3.8/site-packages/morecantile/models.py:849: UserWarning: CRS is no longer part of the GeoJSON specification.Other projection than EPSG:4326 might not be supported.
  warnings.warn(
1357it [00:00, 12139.05it/s]


,geometry,title,grid_name,grid_crs
0,"POLYGON ((856094.717 5942520.327, 856094.717 5...","XYZ tile Tile(x=17084, y=11524, z=15)",WebMercatorQuad,EPSG:3857
1,"POLYGON ((856094.717 5941297.335, 856094.717 5...","XYZ tile Tile(x=17084, y=11525, z=15)",WebMercatorQuad,EPSG:3857
2,"POLYGON ((856094.717 5940074.342, 856094.717 5...","XYZ tile Tile(x=17084, y=11526, z=15)",WebMercatorQuad,EPSG:3857
3,"POLYGON ((856094.717 5938851.350, 856094.717 5...","XYZ tile Tile(x=17084, y=11527, z=15)",WebMercatorQuad,EPSG:3857
4,"POLYGON ((856094.717 5937628.357, 856094.717 5...","XYZ tile Tile(x=17084, y=11528, z=15)",WebMercatorQuad,EPSG:3857
...,...,...,...,...
1352,"POLYGON ((883000.551 5876478.735, 883000.551 5...","XYZ tile Tile(x=17106, y=11578, z=15)",WebMercatorQuad,EPSG:3857
1353,"POLYGON ((883000.551 5875255.742, 883000.551 5...","XYZ tile Tile(x=17106, y=11579, z=15)",WebMercatorQuad,EPSG:3857
1354,"POLYGON ((883000.551 5874032.750, 883000.551 5...","XYZ tile Tile(x=17106, y=11580, z=15)",WebMercatorQuad,EPSG:3857
1355,"POLYGON ((883000.551 5872809.757, 883000.551 5...","XYZ tile Tile(x=17106, y=11581, z=15)",WebMercatorQuad,EPSG:3857


In [8]:
AOI_roads_no_forest.to_crs(epsg=3857,inplace=True)
AOI_roads_no_forest.rename(columns={'FID': 'id_aoi'},inplace=True)

In [9]:
test_crs(epsg3857_tiles_gdf.crs,AOI_roads_no_forest.crs)


tiles_in_restricted_aoi=gpd.sjoin(epsg3857_tiles_gdf,AOI_roads_no_forest, how='inner')

In [10]:
tiles_in_restricted_aoi

,geometry,title,grid_name,grid_crs,index_right,id_aoi
1,"POLYGON ((856094.717 5941297.335, 856094.717 5...","XYZ tile Tile(x=17084, y=11525, z=15)",WebMercatorQuad,EPSG:3857,10,10
2,"POLYGON ((856094.717 5940074.342, 856094.717 5...","XYZ tile Tile(x=17084, y=11526, z=15)",WebMercatorQuad,EPSG:3857,10,10
3,"POLYGON ((856094.717 5938851.350, 856094.717 5...","XYZ tile Tile(x=17084, y=11527, z=15)",WebMercatorQuad,EPSG:3857,10,10
4,"POLYGON ((856094.717 5937628.357, 856094.717 5...","XYZ tile Tile(x=17084, y=11528, z=15)",WebMercatorQuad,EPSG:3857,10,10
5,"POLYGON ((856094.717 5936405.365, 856094.717 5...","XYZ tile Tile(x=17084, y=11529, z=15)",WebMercatorQuad,EPSG:3857,10,10
...,...,...,...,...,...,...
1277,"POLYGON ((881777.558 5896046.614, 881777.558 5...","XYZ tile Tile(x=17105, y=11562, z=15)",WebMercatorQuad,EPSG:3857,74,74
1278,"POLYGON ((881777.558 5894823.621, 881777.558 5...","XYZ tile Tile(x=17105, y=11563, z=15)",WebMercatorQuad,EPSG:3857,74,74
1294,"POLYGON ((881777.558 5875255.742, 881777.558 5...","XYZ tile Tile(x=17105, y=11579, z=15)",WebMercatorQuad,EPSG:3857,73,73
1295,"POLYGON ((881777.558 5874032.750, 881777.558 5...","XYZ tile Tile(x=17105, y=11580, z=15)",WebMercatorQuad,EPSG:3857,71,71


In [11]:
tiles_in_restricted_aoi.drop_duplicates('geometry',inplace=True)

,geometry,title,grid_name,grid_crs,index_right,id_aoi
1,"POLYGON ((856094.717 5941297.335, 856094.717 5...","XYZ tile Tile(x=17084, y=11525, z=15)",WebMercatorQuad,EPSG:3857,10,10
2,"POLYGON ((856094.717 5940074.342, 856094.717 5...","XYZ tile Tile(x=17084, y=11526, z=15)",WebMercatorQuad,EPSG:3857,10,10
3,"POLYGON ((856094.717 5938851.350, 856094.717 5...","XYZ tile Tile(x=17084, y=11527, z=15)",WebMercatorQuad,EPSG:3857,10,10
4,"POLYGON ((856094.717 5937628.357, 856094.717 5...","XYZ tile Tile(x=17084, y=11528, z=15)",WebMercatorQuad,EPSG:3857,10,10
5,"POLYGON ((856094.717 5936405.365, 856094.717 5...","XYZ tile Tile(x=17084, y=11529, z=15)",WebMercatorQuad,EPSG:3857,10,10
...,...,...,...,...,...,...
1255,"POLYGON ((881777.558 5922952.448, 881777.558 5...","XYZ tile Tile(x=17105, y=11540, z=15)",WebMercatorQuad,EPSG:3857,76,76
1259,"POLYGON ((881777.558 5918060.478, 881777.558 5...","XYZ tile Tile(x=17105, y=11544, z=15)",WebMercatorQuad,EPSG:3857,75,75
1318,"POLYGON ((883000.551 5918060.478, 883000.551 5...","XYZ tile Tile(x=17106, y=11544, z=15)",WebMercatorQuad,EPSG:3857,75,75
1319,"POLYGON ((883000.551 5916837.485, 883000.551 5...","XYZ tile Tile(x=17106, y=11545, z=15)",WebMercatorQuad,EPSG:3857,75,75


In [29]:
tiles_in_restricted_aoi.drop(columns=['grid_name', 'grid_crs', 'index_right', 'id_aoi'],inplace=True)
tiles_in_restricted_aoi.reset_index(drop=True, inplace=True)

In [36]:
[x+', '+y+', '+z for x, y, z in xyz]

['17084, 11525, 15',
 '17084, 11526, 15',
 '17084, 11527, 15',
 '17084, 11528, 15',
 '17084, 11529, 15',
 '17084, 11530, 15',
 '17084, 11533, 15',
 '17084, 11534, 15',
 '17084, 11538, 15',
 '17084, 11539, 15',
 '17084, 11541, 15',
 '17084, 11544, 15',
 '17084, 11545, 15',
 '17084, 11547, 15',
 '17084, 11548, 15',
 '17084, 11549, 15',
 '17084, 11550, 15',
 '17084, 11552, 15',
 '17084, 11554, 15',
 '17084, 11561, 15',
 '17084, 11562, 15',
 '17084, 11563, 15',
 '17085, 11525, 15',
 '17085, 11526, 15',
 '17085, 11527, 15',
 '17085, 11528, 15',
 '17085, 11529, 15',
 '17085, 11530, 15',
 '17085, 11531, 15',
 '17085, 11532, 15',
 '17085, 11533, 15',
 '17085, 11534, 15',
 '17085, 11535, 15',
 '17085, 11536, 15',
 '17085, 11537, 15',
 '17085, 11538, 15',
 '17085, 11539, 15',
 '17085, 11540, 15',
 '17085, 11541, 15',
 '17085, 11543, 15',
 '17085, 11544, 15',
 '17085, 11545, 15',
 '17085, 11547, 15',
 '17085, 11548, 15',
 '17085, 11549, 15',
 '17085, 11550, 15',
 '17085, 11551, 15',
 '17085, 1155

In [37]:
xyz=[]
for idx in tiles_in_restricted_aoi.index:
    xyz.append([re.sub('[^0-9]','',coor) for coor in tiles_in_restricted_aoi.loc[idx,'title'].split(',')])

tiles_in_restricted_aoi['id'] = [x+', '+y+', '+z for x, y, z in xyz]
    
print(tiles_in_restricted_aoi)

                                               geometry  \
0     POLYGON ((856094.717 5941297.335, 856094.717 5...   
1     POLYGON ((856094.717 5940074.342, 856094.717 5...   
2     POLYGON ((856094.717 5938851.350, 856094.717 5...   
3     POLYGON ((856094.717 5937628.357, 856094.717 5...   
4     POLYGON ((856094.717 5936405.365, 856094.717 5...   
...                                                 ...   
1026  POLYGON ((881777.558 5896046.614, 881777.558 5...   
1027  POLYGON ((881777.558 5894823.621, 881777.558 5...   
1028  POLYGON ((881777.558 5875255.742, 881777.558 5...   
1029  POLYGON ((881777.558 5874032.750, 881777.558 5...   
1030  POLYGON ((881777.558 5874032.750, 881777.558 5...   

                                      title                id  
0     XYZ tile Tile(x=17084, y=11525, z=15)  17084, 11525, 15  
1     XYZ tile Tile(x=17084, y=11526, z=15)  17084, 11526, 15  
2     XYZ tile Tile(x=17084, y=11527, z=15)  17084, 11527, 15  
3     XYZ tile Tile(x=17084, y=1152

## Saving files

In [31]:
non_forest_roads.drop(columns=['UUID','GDB-Code'],inplace=True)
non_forest_roads.to_file("/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/non_forest_wide_roads.shp")

In [ ]:
# AOI_roads_poly = gpd.GeoSeries([x for x in AOI_roads_no_forest.geoms], crs=roads.crs)

AOI_roads_no_forest.to_file("02_Data/processed/shapefiles_gpkg/restricted_AOI.shp")

In [ ]:
array=tiles_in_restricted_aoi.index.duplicated()
for k in range(len(array.tolist())):
    if array[k]:
        print(k)

In [36]:
AOI_roads_no_forest.columns

Index(['FID', 'geometry'], dtype='object')

In [42]:
# Save in json format for futur use
tiles_in_restricted_aoi.to_file(TILES_AOI, driver='GeoJSON')

# Save in gpkg for information
layername="epsg3857_z" + str(ZOOM_LEVEL) + "_tiles"
tiles_in_restricted_aoi.to_file(GPKG_TILES, driver='GPKG',layer=layername)


In [11]:
import requests

XYZ_url_completed='https://titiler.vm-gpu-01.stdl.ch/mosaicjson/tiles/18/136704/92313.tif'

param={'url':'/data/mosaic_3857.json','nodata':0,'pixel_selection':'lowest','return_mask':'false'}

r = requests.get(XYZ_url_completed, params=param, allow_redirects=True, verify=False)

/home/gsalamin/Python_env/road_surfaces/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'titiler.vm-gpu-01.stdl.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
type(r.content)

bytes